In [29]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
from urllib.request import urlopen
import re
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools



Schwab_URL = "https://www.schwabjobs.com/List/Custom/Schwab-Job-Category"

stateList = []

count = 1



def scrape(postingUrl,bank):
    print(postingUrl)
    global count
    headers = ["LINK_OF_THE_JOB_FROM_BANKS_WEBSITE", "JOB_TITLE/JOB_ROLE", "JOB_DESCRIPTION", "BANK_NAME"]
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        request = urllib2.Request(postingUrl,headers=hdr)
        html_page = urlopen(request)
        soup2 =  BeautifulSoup(html_page,"html.parser")
        value = soup2.findAll("div", {"class": "desc"})
        title = soup2.find("div", {"class":"row render"}).h1.text
        description = ""
        for val in value:
            description += val.text
    
        values = [postingUrl, title, description, bank]
     
    except:
        print("Exception")
        return ("An exception occurred")
    
    return dict(zip(headers, values))

def scrape_pages_schwab(pages):
    stateList = {}
    for p in pages:
        pageHtml = urlopen(p)
        soup = BeautifulSoup(pageHtml)
        table = soup.find('table', 'JobListTable')
        pageDataframe = pd.DataFrame([])
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = Schwab_BASE + link.get('href')
            pageDict = scrape(page,"Schwab")
            if (type(pageDict) is dict):
                data = pd.DataFrame([pageDict])
                # if file does not exist write header 
                if not os.path.isfile('schwabData_description.csv'):
                    print("writing to csv")
                    data.to_csv('schwabData_description.csv', header='column_names')
                else: # else it exists so append without writing the header
                    data.to_csv('schwabData_description.csv', mode='a', header=False)
        

        

def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(Schwab_BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(Schwab_BASE + key)
    return pageLinks

def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []
        cat = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
                cat.append(tag.text)
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        #categCsv = dict(zip(cat, numPages))
        #plt.bar(range(len(categCsv)), list(categCsv.values()), align='center')
        #plt.xticks(range(len(categCsv)), list(categCsv.keys()))
        #plt.xticks(rotation = 90)
        #plt.imshow()
        return dict(zip(catLinks, numPages))

def scrape_site(Url):
    scrape_page_pnc(FIRST_LISTING_PAGE)
    for i in range(12,996,12):
        nextPageUrl = LISTING_URL_PREFIX +  str(i) + "&" + LISTING_URL_POSTFIX
        scrape_page_pnc(nextPageUrl)
        break

def get_data(bank,Url):
    if (bank == "Schwab"):
        res = urlopen(Url)
        categories = get_category(res)
        pages = get_pages(categories)
        scrape_pages_schwab(pages)

if __name__ == '__main__':
    get_data("Schwab","https://www.schwabjobs.com/List/Custom/Schwab-Job-Category")
    #scrape("https://www.schwabjobs.com/ShowJob/Id/1887293/Estate-Services-Relationship-Professional-Westlake-TX/","Schwab")

https://www.schwabjobs.com/ShowJob/Id/1707369/Treasury-Sr.-Specialist-Manager-–-Liquidity-Risk-Management/
Exception
https://www.schwabjobs.com/ShowJob/Id/1941909/Managing-Director,-CRE-Finance/
https://www.schwabjobs.com/ShowJob/Id/1919260/Sr.-Manager,-Treasury/
https://www.schwabjobs.com/ShowJob/Id/1840064/Senior-Team-Manager,-Technology-Procurement,-Vendor-Sourcing-Management/
https://www.schwabjobs.com/ShowJob/Id/1952521/Sr.-Specialist,-Finance/
https://www.schwabjobs.com/ShowJob/Id/1842074/Sr.-Manager,-Business-Systems-Analyst/
https://www.schwabjobs.com/ShowJob/Id/1863067/Sr.-Manager,-Finance/
https://www.schwabjobs.com/ShowJob/Id/1949850/Valuation-Oversight/
https://www.schwabjobs.com/ShowJob/Id/1949474/Manager,-Finance/
https://www.schwabjobs.com/ShowJob/Id/1947943/Deal-Desk-Sourcing-Manager/
https://www.schwabjobs.com/ShowJob/Id/1939618/Deal-Desk-Sourcing-Manager/
https://www.schwabjobs.com/ShowJob/Id/1677626/Senior-Team-Manager,-Finance/
https://www.schwabjobs.com/ShowJob/Id/

https://www.schwabjobs.com/ShowJob/Id/1949780/Brokerage-Service-Specialist-Institutional/
https://www.schwabjobs.com/ShowJob/Id/1949779/Technical-Client-Service-Specialist/
https://www.schwabjobs.com/ShowJob/Id/1948352/Client-Relationship-Specialist-New-Orleans,-LA/
https://www.schwabjobs.com/ShowJob/Id/1947946/Client-Relationship-Specialist-The-Woodlands,,-TX/
https://www.schwabjobs.com/ShowJob/Id/1946885/Regional-Support-Specialist,-Advisor-Services-(Centralized-Region)/
https://www.schwabjobs.com/ShowJob/Id/1769918/Financial-Consultant-Academy-–-Indianapolis,-IN-–-Summer-2019/
Exception
https://www.schwabjobs.com/ShowJob/Id/1925338/Specialist/
https://www.schwabjobs.com/ShowJob/Id/1864570/Relationship-Specialist,-Advisor-Custody-Trading/
https://www.schwabjobs.com/ShowJob/Id/1942847/Client-Relationship-Specialist-Denver,-CO/
https://www.schwabjobs.com/ShowJob/Id/1942676/Legal-Counsel-Digital,-Technology-Transactions-Cybersecurity/
https://www.schwabjobs.com/ShowJob/Id/1941986/Client

https://www.schwabjobs.com/ShowJob/Id/1908554/Associate-Financial-Consultant-Woodland-Hills,-CA/
https://www.schwabjobs.com/ShowJob/Id/1916837/VP,-Financial-Consultant-Fairfield,-CT/
https://www.schwabjobs.com/ShowJob/Id/1906856/Associate-Financial-Consultant-Richmond,-VA/
https://www.schwabjobs.com/ShowJob/Id/1909297/VP,-Financial-Consultant-Novi,-MI/
https://www.schwabjobs.com/ShowJob/Id/1866266/Associate-Financial-Consultant-St-Paul/
https://www.schwabjobs.com/ShowJob/Id/1920849/VP,-Financial-Consultant-Providence,-RI/
https://www.schwabjobs.com/ShowJob/Id/1919853/VP,-Financial-Consultant-Philadelphia,-PA/
https://www.schwabjobs.com/ShowJob/Id/1906710/VP,-Financial-Consultant-–-Richmond,-VA/
Exception
https://www.schwabjobs.com/ShowJob/Id/1743541/VP,-Financial-Consultant-–-Virginia-Beach,-VA/
Exception
https://www.schwabjobs.com/ShowJob/Id/1879091/Financial-Consultant-Partner-Towson,-MD/
https://www.schwabjobs.com/ShowJob/Id/1953162/VP,-Financial-Consultant-Greenville,-SC/
https://w

https://www.schwabjobs.com/ShowJob/Id/1802517/Associate-Software-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1821025/Staff-Threat-Management-Assessment-Analyst/
https://www.schwabjobs.com/ShowJob/Id/1593302/Sr.-Configuration-Release-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1753592/Sr.-Staff-Application-Security-Analyst/
https://www.schwabjobs.com/ShowJob/Id/1922114/Sr.-Staff,-Scrum-Master/
https://www.schwabjobs.com/ShowJob/Id/1956597/Sr.-Analyst-–-Cyber-Risk-Intelligence-Reporting/
Exception
https://www.schwabjobs.com/ShowJob/Id/1851211/Managing-Director,-Software-Applications-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1885656/Technical-Director-Big-Data-Strategy-Architect/
https://www.schwabjobs.com/ShowJob/Id/1922626/ITSM-Configuration-Analyst/
https://www.schwabjobs.com/ShowJob/Id/1922237/Sr.-Business-Analyst/
https://www.schwabjobs.com/ShowJob/Id/1889098/Sr.-Business-Intelligence-Analyst/
https://www.schwabjobs.com/ShowJob/Id/1889573/Sr.-Staff-Business-Systems-Anal

https://www.schwabjobs.com/ShowJob/Id/1797748/Solution-Architect-Marketing-Data/
https://www.schwabjobs.com/ShowJob/Id/1671989/Sr.-Big-Data-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1829724/Staff-iOS-Developer/
https://www.schwabjobs.com/ShowJob/Id/1943644/Technical-Director-Software-Architect-(.NET)/
https://www.schwabjobs.com/ShowJob/Id/1943539/Senior-C#-.NET-Developer/
https://www.schwabjobs.com/ShowJob/Id/1871697/Technical-Director-DevOps-Architect-(ERN4)/
https://www.schwabjobs.com/ShowJob/Id/1943209/Sr-Manager-Data-Engineering/
https://www.schwabjobs.com/ShowJob/Id/1832389/Sr.-Manager,-QA-Engineering/
https://www.schwabjobs.com/ShowJob/Id/1942675/Senior-Manager-–-Big-Data-System-Support-Engineer/
Exception
https://www.schwabjobs.com/ShowJob/Id/1942476/Senior-Staff,-Enterprise-Architecture/
https://www.schwabjobs.com/ShowJob/Id/1871700/MOS-System-Platform-Owner-–-Client-Verification-and-Risk-Engine/
Exception
https://www.schwabjobs.com/ShowJob/Id/1800717/Sr.-Software-Enginee

https://www.schwabjobs.com/ShowJob/Id/1954917/Internal-Audit-Senior-Team-Manager,-Technology/
https://www.schwabjobs.com/ShowJob/Id/1792073/Internal-Audit-Senior-Manager,-Quality-Assurance/
https://www.schwabjobs.com/ShowJob/Id/1948573/Internal-Audit-Managing-Director,-Professional-Practices/
https://www.schwabjobs.com/ShowJob/Id/1913171/Internal-Audit-Manager,-Data-Governance/
https://www.schwabjobs.com/ShowJob/Id/1762312/Internal-Audit-Senior,-Project-Risk/
https://www.schwabjobs.com/ShowJob/Id/1913173/Internal-Audit-Senior-Specialist,-Data-Analytics/
https://www.schwabjobs.com/ShowJob/Id/1932082/Internal-Audit-Senior,-Broker-Dealer-Asset-Management/
https://www.schwabjobs.com/ShowJob/Id/1946260/Internal-Audit-Senior-Specialist,-Professional-Practices/
https://www.schwabjobs.com/ShowJob/Id/1946256/Internal-Audit-Manager,-Risk-Assessment-Data/
https://www.schwabjobs.com/ShowJob/Id/1940275/Sr-Specialist,-Internal-Audit/
https://www.schwabjobs.com/ShowJob/Id/1936384/Internal-Audit-Manag

https://www.schwabjobs.com/ShowJob/Id/1888407/Manager-Specialist,-QA-QC-Model-Risk-Oversight/
https://www.schwabjobs.com/ShowJob/Id/1871128/Senior-Manager,-Independent-Branch-Services-Supervision/
https://www.schwabjobs.com/ShowJob/Id/1904746/Risk-Analyst,-First-Line-Control-Testing/
https://www.schwabjobs.com/ShowJob/Id/1904733/Sr.-Risk-Analyst,-First-Line-Control-Testing-Chicago-IL/
https://www.schwabjobs.com/ShowJob/Id/1803348/Sr-Mgr,-Effective-Challenge-Issues-Mgt/
https://www.schwabjobs.com/ShowJob/Id/1923407/Sr.-Business-Consultant,-Advisor-Services-Business-Consulting/
https://www.schwabjobs.com/ShowJob/Id/1957752/Vice-President,-Branch-Manager-Chestnut-Hill,-MA/
https://www.schwabjobs.com/ShowJob/Id/1950451/Business-Development-Support-Specialist,-Advisor-Services-(Central-Region)/
https://www.schwabjobs.com/ShowJob/Id/1865260/Vice-President,-Assistant-Branch-Manager-Millburn,-NJ/
https://www.schwabjobs.com/ShowJob/Id/1956601/Vice-President,-Branch-Manager-San-Francisco,-CA-(Ca